In [22]:
import pandas as pd
import numpy as np
from scipy import stats

In [9]:
data = pd.read_csv('stack-overflow-developer-survey-2022/survey_results_public.csv')
data = data.dropna(subset=['DevType'])

In [10]:
def job_cluster(x):
    # if student/other in multiple type answer, delete student/other
    if 'Student' in x:
        x = x.replace('Student;', '')
        x = x.replace(';Student', '')
    if 'Other (please specify):' in x:
        x = x.replace('Other (please specify):;', '')
        x = x.replace(';Other (please specify):', '')

    
    # order of processing is important (from cross-domain to specific)
    if 'Educator' in x:
        return 'Educator'
    elif any(word in x for word in ['manager', 'Executive']):
        return 'Manager'
    elif any(word in x for word in ['Data ', 'data', 'Marketing']):
        return 'Data'
    elif 'full-stack' in x:
        return 'Developer FS'
    elif 'game' in x:
        return 'Game Dev'
    elif any(word in x for word in ['front-end', 'Designer']):
        return 'Developer FE'
    elif 'back-end' in x:
        return 'Developer BE'
    elif 'Developer,' in x:
        return 'Testing/Apps/mobile'
    elif 'admin' in x:
        return 'Admin'
    elif any(word in x for word in ['DevOps', 'reliability']):
        return 'DevOps'
    elif any(word in x for word in ['Security', 'Blockchain', 'Cloud']):
        return 'Security/Cloud'
    elif any(word in x for word in ['researcher', 'Scientist']):
        return 'Researching'
    else:
        return x

data['TYPE'] = data['DevType'].apply(job_cluster)

In [17]:
devs = data[['TYPE','MainBranch', 'Country', 'ConvertedCompYearly', 'Employment']]
devs.head(1)
devs.to_csv('rudi.csv')

In [24]:
sals = devs.groupby(['Country']).agg({'ConvertedCompYearly':np.mean, 'TYPE':stats.mode})
sals

/home/mabioo/.local/lib/python3.10/site-packages/pandas/core/groupby/groupby.py:1649: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  f = lambda x: func(x, *args, **kwargs)
/home/mabioo/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


,ConvertedCompYearly,TYPE
Country,,
Afghanistan,33914.666667,"([Developer FS], [14])"
Albania,24895.148148,"([Developer FS], [20])"
Algeria,12732.888889,"([Developer FS], [10])"
Andorra,98003.125000,"([Developer BE], [3])"
Angola,6222.750000,"([Developer FS], [4])"
...,...,...
"Venezuela, Bolivarian Republic of...",17079.400000,"([Developer FS], [37])"
Viet Nam,38093.537634,"([Developer FS], [76])"
Yemen,17464.666667,"([Developer FS], [4])"


In [7]:
def splitter(x: str):
    return x.split('(')[0].rstrip()

def encode_multichoices(column):
    df = pd.DataFrame(data[column])
    df = df[column].str.get_dummies(sep=';')
    df = df.drop(['Other (Please specify):'], axis=1, errors='ignore')
    df = df.drop(['Other (please specify):'], axis=1, errors='ignore')
    df.columns = list(map(splitter, df.columns))  
    # df = df.add_prefix(f'{column}_')
    return df

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
from dython.nominal import associations
for col in data.drop(['Unnamed: 0', 'YearsCode', 'ConvertedCompYearly'], axis=1).columns:
    learn = encode_multichoices(col)
    X = learn.reset_index(drop=True)
    y = data['TYPE'].reset_index(drop=True)

    # heatmaps for checking for multicoll
    associations(pd.concat((X, y) ,axis=1), figsize=(10,10), fmt='.1f', title=col) 

AttributeError: Can only use .str accessor with string values!

In [18]:
data.drop(['YearsCode', 'ConvertedCompYearly'], axis=1)

,Unnamed: 0,Full in-person,Fully remote,Hybrid,Associate degree,Bachelor’s degree,Master’s degree,Other doctoral degree,Primary/elementary school,Professional degree,...,People manager,Automated testing,Continuous integration,DevOps function,Developer portal or other central places to find tools/services,Innersource initiative,Microservices,None of these,Observability tools,TYPE
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,1,1,0,1,0,1,Developer BE
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Manager
2,2,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,Developer FS
3,3,0,0,1,0,1,0,0,0,0,...,0,1,1,1,0,1,0,0,1,Manager
4,4,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Developer FE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35756,35756,0,1,0,0,1,0,0,0,0,...,0,1,1,1,1,0,1,0,1,Developer FS
35757,35757,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Developer BE
35758,35758,0,1,0,0,0,1,0,0,0,...,0,1,1,1,1,0,1,0,1,Developer BE
35759,35759,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,Developer BE
